<a href="https://colab.research.google.com/github/XinzhouLi/Toxic_Language_Detection_in_Social_Media/blob/main/LSTM_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ! git clone https://github.com/XinzhouLi/Toxic_Language_Detection_in_Social_Media.git
# ! pip install torch==1.8 torchtext==0.9

 https://www.cnblogs.com/cxq1126/p/13466998.html
 这篇文章 有讲述如何怎么建立完整的训练过程


 https://www.cnblogs.com/cxq1126/p/13504437.html
 这个就是我们的最终目标 LSTM加上Attention


 https://www.cnblogs.com/cxq1126/p/13508696.html
 这个是transformer的文本分类我还没看先扔这



In [2]:
import torch
import pandas as pd
import numpy as np
from torch.utils import data
from torch import nn, optim
import torchtext
import re
import sys


d:\progarmSoftware\Miniconda\envs\3.8\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(torch.__version__)
# print(sys.version)
print(torchtext.__version__)
print(torch.version.cuda)

1.8.2
0.9.2
11.1


In [4]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(torch.cuda.is_available())

True


In [5]:
def unify_format(text):
    return re.sub(r"[^a-zA-Z0-9]", " ", text).lower()

In [6]:
# implement dataset extends util.data.Dataset
class ToxicDataset(data.Dataset):
    def __init__(self, *args):
        super().__init__()
        # Read data in
        if len(args) == 1:
            # for initialize training dataset
            filepath = args[0]
            self.dataframe = pd.read_csv(
                filepath, iterator=True, header=0, encoding='utf-8', usecols=['comment_text', 'toxic'])
            # initialize pandas dataframe to store the data
            self.dataframe = pd.DataFrame(self.dataframe.read())
            self.dataframe.convert_dtypes()
            # Clean the data using regular expersion, only reserve letter and number
            self.dataframe['comment_text'] = self.dataframe['comment_text'].apply(unify_format)
            self.dataframe.to_csv('train_.csv', index=False, header=["text", "label"])
        else:
            # for initialize testing dataset
            filepath1 = args[0]
            filepath2 = args[1]
            test_comments = pd.DataFrame(pd.read_csv(
                filepath1, iterator=True, header=0, usecols=['comment_text', 'id']).read())
            test_label = pd.DataFrame(pd.read_csv(
                filepath2, iterator=True, header=0, usecols=['id', 'toxic']).read())
            # merge two dataframe together by unique key id
            result = pd.merge(test_comments, test_label, how='left', on=['id'])
            # select the text and label col
            self.dataframe = result[result['toxic']>= 0].loc[:, ['comment_text', 'toxic']]
            self.dataframe.convert_dtypes()
            # Clean the data using regular expersion, only reserve letter and number
            self.dataframe['comment_text'] = self.dataframe['comment_text'].apply(unify_format)
            self.dataframe.to_csv('test_.csv', index=False, header=["text", "label"])
            

    # override the getiem function, return tuple contains comment and label

    def __getitem__(self, index):
        comment, label = self.dataframe.iat[index, 0], self.dataframe.iat[index, 1]
        return (comment, label)

    # Override the len function, return the length of the dataframe
    def __len__(self):
        return len(self.dataframe.loc[:, ['comment_text']])


In [7]:
# init dataset
# train_dataset = ToxicDataset("train.csv")
# test_dataset = ToxicDataset("test.csv", "test_labels.csv")
# train_dataset = ToxicDataset("/content/Toxic_Language_Detection_in_Social_Media/train.csv")
# test_dataset = ToxicDataset("/content/Toxic_Language_Detection_in_Social_Media/test.csv", "/content/Toxic_Language_Detection_in_Social_Media/test_labels.csv")

In [8]:
def get_dataset(csv_data, text_field, label_field, test=False):
    fields = [("id", None), # we won't be needing the id, so we pass in None as the field
                 ("text", text_field), ("label", label_field)]       
    examples = []

    if test:
        # 如果为测试集，则不加载label
        for text in csv_data['text']:
            examples.append(torchtext.legacy.data.Example.fromlist([None, text, None], fields))
    else:
        for text, label in zip(csv_data['text'], csv_data['label']):
            examples.append(torchtext.legacy.data.Example.fromlist([None, text, label], fields))
    return examples, fields

In [9]:
train_data = pd.read_csv('train_.csv')
test_data = pd.read_csv("test.csv")
tokenize = lambda x: x.split()
TEXT = torchtext.legacy.data.Field(sequential=True, tokenize=tokenize, lower=True)
LABEL = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

# 得到构建Dataset所需的examples和fields
train_examples, train_fields = get_dataset(train_data, TEXT, LABEL)
test_examples, test_fields = get_dataset(train_data, TEXT, LABEL)
# 构建Dataset数据集
train = torchtext.legacy.data.Dataset(train_examples, train_fields)
test = torchtext.legacy.data.Dataset(test_examples, test_fields)

In [10]:
print(train[0].__dict__.keys())
print(train[0].text)
print(train[6].label)

dict_keys(['text', 'label'])
['explanation', 'why', 'the', 'edits', 'made', 'under', 'my', 'username', 'hardcore', 'metallica', 'fan', 'were', 'reverted', 'they', 'weren', 't', 'vandalisms', 'just', 'closure', 'on', 'some', 'gas', 'after', 'i', 'voted', 'at', 'new', 'york', 'dolls', 'fac', 'and', 'please', 'don', 't', 'remove', 'the', 'template', 'from', 'the', 'talk', 'page', 'since', 'i', 'm', 'retired', 'now', '89', '205', '38', '27']
1


In [11]:
TEXT.build_vocab(train, max_size=10000, vectors='glove.6B.300d',)
LABEL.build_vocab(train)

In [12]:
print(TEXT)

In [13]:
class BLSTMModel(nn.Module):
    def __init__(self, vocab_size, embeding_dim, hidden_dim):
        super().__init__()
        # implement embedding function to convert text to numerical data
        self.embedding = nn.Embedding(vocab_size, embeding_dim)
        self.rnn = nn.LSTM(embeding_dim, hidden_dim, num_layers =2, bidirectional=True, dropout = 0.5)
        self.fc = nn.Linear(hidden_dim*2,1)
        self.dropout = nn.Dropout(0.5)
    def forward(self, x):

        embediding = self.dropout(self.embedding(x))
        output,(hidden, cell) = self.rnn(embediding)
        hidden = torch.cat([hidden[-2], hidden[-1]],dim=1)
        hidden = self.dropout(hidden)
        out = self.fc(hidden)
        return out


In [14]:
train_dataloader = torchtext.legacy.data.BucketIterator(dataset=train, batch_size=180, shuffle=True)#, sort_within_batch=False, sort_key=lambda x: len(x.text), repeat=False)
test_dataloader = torchtext.legacy.data.BucketIterator(dataset=test, batch_size=180, shuffle=True)

In [15]:
# init Lstm model 
BLSTM = BLSTMModel(len(TEXT.vocab), 300, 256)
# replace the embedding to the pretrained embedding Glove
pretrained = TEXT.vocab.vectors
BLSTM.embedding.weight.data.copy_(pretrained) 

optimizer = optim.Adam(BLSTM.parameters(), lr=1e-3)
criteon = nn.BCEWithLogitsLoss().to(DEVICE)
BLSTM = BLSTM.to(DEVICE)

In [16]:
# training process
# 没写完
def binary_acc(preds, y):
    """
    get accuracy
    """
    preds = torch.round(torch.sigmoid(preds))
    correct = torch.eq(preds, y).float()
    acc = correct.sum() / len(correct)
    return acc

def train(rnn, dataloader, optimizer, criteon):
    
    avg_acc = []
    rnn.train()
    i=0
    for data in dataloader:
        text = data.text
        text = text.to(DEVICE)
        label = data.label
        label = label.to(DEVICE)
        # [seq, b] => [b, 1] => [b]

        pred = rnn(text).squeeze(1)
        # 
        loss = criteon(pred.float(), label.float())
        acc = binary_acc(pred, label).item()
        avg_acc.append(acc)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i%500 == 0:
            print(i, acc)
        i +=1
        
    avg_acc = np.array(avg_acc).mean()
    print('avg acc:', avg_acc)		
    
def eval(rnn, iterator, criteon):
    
    avg_acc = []
    
    rnn.eval()
    
    with torch.no_grad():
        for data in iterator:
            text = data.text
            text = text.to(DEVICE)
            label = data.label
            label = label.to(DEVICE)
            # [b, 1] => [b]
            pred = rnn(text).squeeze(1)

            #
            loss = criteon(pred.float(), label.float())

            acc = binary_acc(pred, label).item()
            avg_acc.append(acc)
        
    avg_acc = np.array(avg_acc).mean()
    
    print('>>test:', avg_acc)




In [17]:
for epoch in range(5):
    train(BLSTM, train_dataloader, optimizer, criteon)
torch.save(BLSTM.state_dict(), "Model.pth")

0 0.2888889014720917
500 0.9611111283302307
avg acc: 0.9493802320527116
0 0.9722222685813904
500 0.9555555582046509
avg acc: 0.963284778890642
0 0.9777777791023254
500 0.9722222685813904
avg acc: 0.9667297369194676
0 0.9611111283302307
500 0.9611111283302307
avg acc: 0.9685966161488143
0 0.9833333492279053
500 0.9611111283302307
avg acc: 0.9704122875293185


In [18]:
BLSTM.load_state_dict(torch.load("Model.pth"))
eval(BLSTM, test_dataloader, criteon)

>>test: 0.9744206670306312
